In [2]:
import json
import pandas as pd
import glob


In [9]:
len(glob.glob("/home/gujiashe/Tk-Instruct/data/tasks/*.json"))

1613

In [27]:
tasks = {}

tasks['Examples'] = []
tasks['Categories'] = []
paths = []

for task_path in glob.glob("/home/gujiashe/Tk-Instruct/data/tasks/*.json"):
    with open(task_path, "r") as task_f:
        s = task_f.read()
        task = json.loads(s)
        if len(task["Input_language"])>1 or task["Input_language"][0] != "English":
            continue
        if len(task["Output_language"])>1 or task["Output_language"][0] != "English":
            continue
        pos_samples = "Positive Examples: " + '\n\n'
        for example in task['Positive Examples']:
            sample = ""
            for k in example:
                sample += k + ': ' + '\n\n'
                sample += example[k] + '\n\n'
            sample += '\n\n'
            pos_samples += sample

        neg_samples = "Negative Examples: " + '\n\n'
        for example in task['Negative Examples']:
            sample = ""
            for k in example:
                sample += k + ': ' + '\n\n'
                sample += example[k] + '\n\n'
            sample += '\n\n'
            neg_samples += sample
        samples = pos_samples + neg_samples
        Categories = task['Categories'][0]
        tasks['Examples'].append(samples)
        tasks['Categories'].append(Categories)
        paths.append(task_path)
tasks = pd.DataFrame(tasks)

In [26]:
paths[76]

'/home/gujiashe/Tk-Instruct/data/tasks/task896_miam_language_classification.json'

In [29]:
tasks['Examples']

0       Positive Examples: \n\ninput: \n\nI have bough...
1       Positive Examples: \n\ninput: \n\nRoman Yakub ...
2       Positive Examples: \n\ninput: \n\nBob helped h...
3       Positive Examples: \n\ninput: \n\nCategory: EV...
4       Positive Examples: \n\ninput: \n\nstory : Hono...
                              ...                        
1032    Positive Examples: \n\ninput: \n\n5 + 3\n\nout...
1033    Positive Examples: \n\ninput: \n\nText: Abbott...
1034    Positive Examples: \n\ninput: \n\n1, 3, ['238'...
1035    Positive Examples: \n\ninput: \n\nI work 40 ho...
1036    Positive Examples: \n\ninput: \n\nWhich State ...
Name: Examples, Length: 1037, dtype: object

In [40]:
print(tasks['Examples'][384])

Positive Examples: 

input: 

John has y dollars to spend on some new CDs from Music Plus, an online record store. He can buy any CDs at the members' price of x dollars each. To be a member, John has to pay a one-time fee of $19. Which of the following expressions represents the number of CDs John can purchase from Music Plus?
(A)\(\frac{xy}{19}\) (B)\(\frac{y + 19}{x}\) (C)\(\frac{2y - x}{19}\) (D)\(\frac{y - 19}{x}\) (E)\(\frac{19 - x}{y}\)

output: 

D

explanation: 

john must pay 19 dollar to buy a membership then he will have \(y-19\) dollar remaining so he can buy \(rac{y - 19}{x}\) CDs.



input: 

There are four colors of lottery tickets in a hat. If \(\frac { 1 } { 10 } \) of the tickets in the hat are green, \(\frac { 1 } { 2 } \) are white, \(\frac { 1 } { 4 } \) are blue, and the remaining 30 tickets are pink, what is the number of blue tickets in the hat?
(A)25 (B)50 (C)75 (D)120 (E)200

output: 

B

explanation: 

\(rac { 17 } { 20 } \) of the tickets are either green,

In [38]:
for i, ex in enumerate(tasks['Examples']):
    if '🙂' in ex:
        print(i)

74
96
115
321
341
554


In [30]:
tasks.to_csv("tasks.csv", index=None)

In [31]:
pd.read_csv("tasks.csv")

,Examples,Categories
0,Positive Examples: \n\ninput: \n\nI have bough...,Text Matching
1,Positive Examples: \n\ninput: \n\nRoman Yakub ...,Question Answering
2,Positive Examples: \n\ninput: \n\nBob helped h...,Linguistic Probing
3,Positive Examples: \n\ninput: \n\nCategory: EV...,Misc.
4,Positive Examples: \n\ninput: \n\nstory : Hono...,Question Answering
...,...,...
1032,Positive Examples: \n\ninput: \n\n5 + 3\n\nout...,Mathematics
1033,Positive Examples: \n\ninput: \n\nText: Abbott...,Text Categorization
1034,"Positive Examples: \n\ninput: \n\n1, 3, ['238'...",Program Execution
1035,Positive Examples: \n\ninput: \n\nI work 40 ho...,Text Categorization
